In [1]:
import pandas as pd

In [2]:
df_huls = pd.read_pickle('/home/maghsk/w1/3-1.AnalysisHuls/input/df-all-merged-1.pkl.zstd', compression='zstd')
df_label = pd.read_pickle('output/0328-all_url_list.pkl')

In [3]:
print(df_huls.shape)
print(df_label.shape)

(8618, 10)
(8617, 6)


In [4]:
idx = 46
assert df_huls.loc[idx].url == df_label.loc[idx].url and df_huls.loc[idx+1].url != df_label.loc[idx+1].url

In [5]:
df_label.loc[idx+0.5] = ["['awwwards']", "http://aalimentaria.com", 1, None, None, 'Webdriver Error']
df_label = df_label.sort_index().reset_index(drop=True)

In [6]:
print(df_label.loc[idx+1])
df_label[df_huls.url.str.strip() != df_label.url.str.strip()]

category                   ['awwwards']
url             http://aalimentaria.com
ERROR                                 1
usecase                            None
interactions                       None
comments                Webdriver Error
Name: 47, dtype: object


,category,url,ERROR,usecase,interactions,comments
292,['cssdesign'],https://www.marine-i.co.uk,1,<NA>,<NA>,URL error


In [7]:
df_merged = pd.concat([df_huls, df_label[['ERROR', 'usecase', 'interactions', 'comments']].add_prefix('label_')], axis=1)
df_merged.to_pickle('output/df_huls_label-0403.pkl.zstd', compression='zstd')

In [8]:
df_merged.url = df_merged.url.str.strip()

In [9]:
error_urls = []
for x in df_merged[df_merged.url.str[7:].str.contains('://')].url:
    if 'web.archive.org' in x:
        continue
    print(x)
    pos = -x.find('http://', 6) * x.find('https://', 6)
    error_urls.append(x[:pos])
    error_urls.append(x[pos:])
error_urls = sorted(set(error_urls))
error_urls

http://2018.makemepulse.comhttp://www.honigstudios.com
http://astateofwar.org.au/http://www.imjp.co.jp/nenga2016/
http://authentiks.fr/http://taotajima.jp/
http://awards.paris/new-micra-experiencehttps://www.futurelibrary.no/
http://blackrainbow.la/http://astoryabouttrusts.co.uk
http://bosonprotocol.iohttps://www.poppr.be
http://chordsurfer.redbull.com/http://corrupted-time.artinyan.net
http://disrupt.ouiwill.comhttps://www.marine-i.co.uk
http://lgbt.allstateonline.com/http://nclud.com
http://mono.acervo.iohttp://scars.forhonorgame.com
http://operationsamusreturns.comhttp://en.uglerod.ru/
http://sound-of-change.com/http://honda-dct.com
http://tzina.space/http://opencontinents.com/?stars=1
http://uniel.jp/https://nojdigmedmer.tele2.se/en/
http://vodka-a.comhttps://www.skylinefilms.tv
http://www.blancpain-ocean-commitment.comhttp://witnessgotham.com
http://www.espn.com/espn/feature/story/_/id/19045647/we-fans-documentary-official-site-espnhttp://eien.tokyo
http://www.fetedeslumieres.lyon

['http://12vendome.chaumet.com',
 'http://2018.makemepulse.com',
 'http://astateofwar.org.au/',
 'http://astoryabouttrusts.co.uk',
 'http://authentiks.fr/',
 'http://awards.paris/new-micra-experience',
 'http://blackrainbow.la/',
 'http://bosonprotocol.io',
 'http://chordsurfer.redbull.com/',
 'http://clapat.ro/themes/rayden/',
 'http://corrupted-time.artinyan.net',
 'http://disrupt.ouiwill.com',
 'http://eien.tokyo',
 'http://en.uglerod.ru/',
 'http://goggles.wedze.com/winter-2017-2018/en',
 'http://honda-dct.com',
 'http://lgbt.allstateonline.com/',
 'http://loveforiceland.com',
 'http://mainframe.agvr.co',
 'http://mono.acervo.io',
 'http://nclud.com',
 'http://opencontinents.com/?stars=1',
 'http://operationsamusreturns.com',
 'http://rainforest.arkivert.no',
 'http://scars.forhonorgame.com',
 'http://showyourwarrior.wonderwomanfilm.com/',
 'http://sound-of-change.com/',
 'http://taotajima.jp/',
 'http://tomerlerner.com/',
 'http://tzina.space/',
 'http://uix.me',
 'http://uniel.jp

In [10]:
# drop rows which df_merged.label_comments=='URL error'
df_merged.drop(df_merged[df_merged.label_comments=='URL error'].index, inplace=True)
# add error_urls
df_merged2 = pd.concat([df_merged, pd.DataFrame({'url': error_urls, 'gl_info': None, 'label_ERROR': 1, 'label_comments': 'TODO (URL error)'})], ignore_index=True)

# df_merged2['url'] = df_merged2['url'].astype('string')
# df_merged2['category'] = df_merged2['category'].astype('string')
# df_merged2['get_status'] = df_merged2['get_status'].astype('string')
# df_merged2['dom_load_status'] = df_merged2['dom_load_status'].astype('string')
# df_merged2['webgl_status'] = df_merged2['webgl_status'].astype('string')
# df_merged2['hundred_frames_status'] = df_merged2['hundred_frames_status'].astype('string')
# df_merged2['gl_info_status'] = df_merged2['gl_info_status'].astype('string')
# df_merged2['label_usecase'] = df_merged2['label_usecase'].astype('Int64')
# df_merged2['label_interactions'] = df_merged2['label_interactions'].astype('Int64')
# df_merged2['label_comments'] = df_merged2['label_comments'].astype('string')
df_merged2 = df_merged2.astype(dtype={
    'url' : 'string',
    'category' : 'string',
    'get_status' : 'string',
    'dom_load_status' : 'string',
    'webgl_status' : 'string',
    'hundred_frames_status' : 'string',
    'gl_info_status' : 'string',
    'label_usecase' : 'Int64',
    'label_interactions' : 'Int64',
    'label_comments' : 'string',
})
df_merged2.drop(columns=['screenshot_b64'], inplace=True)
print(df_merged2.dtypes)
df_merged2

url                       string
category                  string
get_status                string
dom_load_status           string
webgl_status              string
hundred_frames_status     string
gl_info_status            string
gl_info                   object
duration                 float64
label_ERROR                Int64
label_usecase              Int64
label_interactions         Int64
label_comments            string
dtype: object


,url,category,get_status,dom_load_status,webgl_status,hundred_frames_status,gl_info_status,gl_info,duration,label_ERROR,label_usecase,label_interactions,label_comments
0,http://matchmaker.msi.com/,['cssdesign'],Ok,Ok,TimeoutException,Ok,Ok,"{'accessTime': 1679326337717, 'contextInfo': [...",31.372748,1,<NA>,<NA>,<NA>
1,https://controltheweather.geostorm.movie/,['cssdesign'],Ok,Ok,TimeoutException,Ok,Ok,"{'accessTime': 1679326341483, 'contextInfo': [...",34.843237,0,1,0,<NA>
2,http://1000.chromeexperiments.com,['awwwards'],Ok,Ok,TimeoutException,Ok,Ok,"{'accessTime': 1679326342200, 'contextInfo': [...",35.065929,0,1,1,<NA>
3,http://127.0.0.1:8000/test1/webgl/easy,['custom'],Ok,Ok,Ok,Ok,Ok,"{'accessTime': 1679326337993, 'contextInfo': [...",30.827378,0,1,0,<NA>
4,http://127.0.0.1:8000/test1/webgl/hard,['custom'],Ok,Ok,Ok,Ok,Ok,"{'accessTime': 1679326370599, 'contextInfo': [...",33.153617,0,1,0,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8762,https://yanlinma.com/ninx/,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,NaN,1,<NA>,<NA>,TODO (URL error)
8763,https://yetimotion.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,NaN,1,<NA>,<NA>,TODO (URL error)
8764,https://yuta-abe.com,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,NaN,1,<NA>,<NA>,TODO (URL error)
8765,https://zoccon.me/,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,NaN,1,<NA>,<NA>,TODO (URL error)


In [11]:
# def url_essential(url):
#     if url.startswith('http://'):
#         url = url[7:]
#     elif url.startswith('https://'):
#         url = url[8:]
#     if url.endswith('/'):
#         url = url[:-1]
#     return url

In [12]:
# !url essential!
df_merged3 = df_merged2.copy()
# append '/' if url is not end with '/'
df_merged3['url2'] = df_merged3.url.apply(lambda x: x if x.endswith('/') else x+'/')
# drop duplicated by url, keep the first
df_merged3.drop_duplicates(subset=['url2'], keep='first', inplace=True)
# create new column `url2` by removing `https://` and `http://`
df_merged3['url2'] = df_merged3.url.str.replace('https://', '').str.replace('http://', '')
# drop duplicated by url2, keep the last
df_merged3.drop_duplicates(subset=['url2'], keep='last', inplace=True)
# drop column `url2`
df_merged3.drop(columns=['url2'], inplace=True)

df_merged3

,url,category,get_status,dom_load_status,webgl_status,hundred_frames_status,gl_info_status,gl_info,duration,label_ERROR,label_usecase,label_interactions,label_comments
0,http://matchmaker.msi.com/,['cssdesign'],Ok,Ok,TimeoutException,Ok,Ok,"{'accessTime': 1679326337717, 'contextInfo': [...",31.372748,1,<NA>,<NA>,<NA>
1,https://controltheweather.geostorm.movie/,['cssdesign'],Ok,Ok,TimeoutException,Ok,Ok,"{'accessTime': 1679326341483, 'contextInfo': [...",34.843237,0,1,0,<NA>
2,http://1000.chromeexperiments.com,['awwwards'],Ok,Ok,TimeoutException,Ok,Ok,"{'accessTime': 1679326342200, 'contextInfo': [...",35.065929,0,1,1,<NA>
3,http://127.0.0.1:8000/test1/webgl/easy,['custom'],Ok,Ok,Ok,Ok,Ok,"{'accessTime': 1679326337993, 'contextInfo': [...",30.827378,0,1,0,<NA>
4,http://127.0.0.1:8000/test1/webgl/hard,['custom'],Ok,Ok,Ok,Ok,Ok,"{'accessTime': 1679326370599, 'contextInfo': [...",33.153617,0,1,0,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8736,https://www.marine-i.co.uk,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,NaN,1,<NA>,<NA>,TODO (URL error)
8741,https://www.ngest.com/,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,NaN,1,<NA>,<NA>,TODO (URL error)
8743,https://www.nucleardissent.com/intro,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,NaN,1,<NA>,<NA>,TODO (URL error)
8744,https://www.oro.com/recruit/,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,NaN,1,<NA>,<NA>,TODO (URL error)


In [21]:
df_merged4 = df_merged3.copy()
# sort by url
df_merged4.sort_values(by=['url'], inplace=True)
# reset index
df_merged4.reset_index(drop=False, inplace=True)
# rename old index to `index_old`
df_merged4.rename(columns={'index': 'index_old'}, inplace=True)

# df_merged4[df_merged4.url == 'https://www.marine-i.co.uk']
df_merged4

,index_old,url,category,get_status,dom_load_status,webgl_status,hundred_frames_status,gl_info_status,gl_info,duration,label_ERROR,label_usecase,label_interactions,label_comments
0,2,http://1000.chromeexperiments.com,['awwwards'],Ok,Ok,TimeoutException,Ok,Ok,"{'accessTime': 1679326342200, 'contextInfo': [...",35.065929,0,1,1,<NA>
1,3,http://127.0.0.1:8000/test1/webgl/easy,['custom'],Ok,Ok,Ok,Ok,Ok,"{'accessTime': 1679326337993, 'contextInfo': [...",30.827378,0,1,0,<NA>
2,4,http://127.0.0.1:8000/test1/webgl/hard,['custom'],Ok,Ok,Ok,Ok,Ok,"{'accessTime': 1679326370599, 'contextInfo': [...",33.153617,0,1,0,<NA>
3,5,http://127.0.0.1:8000/test1/webgl/medium,['custom'],Ok,Ok,Ok,Ok,Ok,"{'accessTime': 1679326372954, 'contextInfo': [...",31.669554,0,1,0,<NA>
4,6,http://127.0.0.1:8000/test3/webgl/easy,['custom'],Ok,Ok,Ok,Ok,Ok,"{'accessTime': 1679326372977, 'contextInfo': [...",30.765577,0,1,0,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8406,8585,https://zulu.longines.com/,['awwwards'],Ok,Ok,Ok,Ok,Ok,"{'accessTime': 1679418113188, 'contextInfo': [...",32.774316,0,1,1,<NA>
8407,8586,https://zumi.gucci.com/,"['awwwards', 'cssdesign']",Ok,Ok,Ok,Ok,Ok,"{'accessTime': 1679420416526, 'contextInfo': [...",32.539581,0,1,1,<NA>
8408,8587,https://zunc.studio,['awwwards'],Ok,Ok,TimeoutException,Ok,Ok,"{'accessTime': 1679435451446, 'contextInfo': [...",37.557081,0,3,0,<NA>
8409,8588,https://zuplo.com/,['awwwards'],Ok,Ok,TimeoutException,Ok,Ok,"{'accessTime': 1679414234994, 'contextInfo': [...",33.459205,0,3,0,<NA>


In [26]:
df_merged4['time_first_gl_call'] = df_merged4['gl_info'].apply(lambda x: x['whenUsedWebgl'] is not None if x is not None else False)

In [27]:
df_merged4['manual'] = False
df_merged4['manual'] = df_merged4['manual'].astype('boolean')
# df_merged4.loc[(df_merged4['label_comments'].str.contains('TODO') | df_merged4['label_comments'].str.contains('URL error') | (df_merged4['label_ERROR'] != 0)), 'manual'] = True
df_merged4.loc[(
    df_merged4['label_comments'].str.contains('TODO')
    | df_merged4['label_comments'].str.contains('URL error')
    | (df_merged4['label_ERROR'] > 1)
), 'manual'] = True
df_merged4.to_pickle('output/clean-df_huls_label-0403.pkl.zstd', compression='zstd')
print('用况', df_merged4.label_usecase.value_counts().sort_index(), sep='\n', end='\n\n')
print('手动检查', df_merged4.manual.value_counts().sort_index(), sep='\n', end='\n\n')
print('手动交互', df_merged4.label_interactions.value_counts().sort_index(), sep='\n', end='\n\n')

用况
1    3075
2      25
3    4128
4      42
Name: label_usecase, dtype: Int64

手动检查
False    8163
True      248
Name: manual, dtype: Int64

手动交互
0    6544
1     774
Name: label_interactions, dtype: Int64



In [28]:
df_merged4.label_comments.value_counts()

scroll                                 282
TODO                                   205
domain                                 105
TODO (URL error)                        39
File not found                          38
white                                    9
CMB                                      5
404                                      4
game                                     3
不交互，不渲染                                  2
scroll/drag                              2
dup                                      2
drag                                     2
tf                                       2
move mouse                               1
ubuntu                                   1
burning affects                          1
no canvas                                1
It works!                                1
facebook                                 1
https://experiments.withgoogle.com/      1
scary!                                   1
mobile                                   1
wait       

In [29]:
df_merged5 = df_merged4.copy()
df_merged5.drop(columns=['gl_info'], inplace=True)
df_merged5.to_excel('output/0403-clean-df_huls_label.xlsx')